## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lebu,CL,-37.6167,-73.6500,59.02,76,56,11.99,broken clouds
1,1,Nelson Bay,AU,-32.7167,152.1500,60.01,77,11,6.91,few clouds
2,2,San Vicente,AR,-26.6167,-54.1333,76.10,59,100,1.05,moderate rain
3,3,Vaini,TO,-21.2000,-175.2000,75.20,88,20,5.75,few clouds
4,4,Kruisfontein,ZA,-34.0033,24.7314,64.99,84,100,1.99,overcast clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,San Vicente,AR,-26.6167,-54.1333,76.10,59,100,1.05,moderate rain
3,3,Vaini,TO,-21.2000,-175.2000,75.20,88,20,5.75,few clouds
5,5,Rikitea,PF,-23.1203,-134.9692,77.94,74,74,7.94,light rain
10,10,Saint-Philippe,RE,-21.3585,55.7679,75.20,83,0,4.61,light rain
29,29,Warri,NG,5.5167,5.7500,79.90,88,76,5.75,broken clouds
30,30,Butaritari,KI,3.0707,172.7902,82.22,74,100,9.22,overcast clouds
31,31,Puerto Ayora,EC,-0.7393,-90.3518,80.01,89,75,8.05,light rain
32,32,Brazzaville,CG,-4.2658,15.2832,75.20,94,20,3.44,light rain
33,33,Honavar,IN,14.2833,74.4500,81.97,78,50,3.71,scattered clouds
38,38,Nioro,GM,13.3500,-15.7500,84.83,35,100,12.10,overcast clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                228
City                   228
Country                225
Lat                    228
Lng                    228
Max Temp               228
Humidity               228
Cloudiness             228
Wind Speed             228
Current Description    228
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
#clean_df.count()

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,San Vicente,AR,76.10,moderate rain,-26.6167,-54.1333,
3,Vaini,TO,75.20,few clouds,-21.2000,-175.2000,
5,Rikitea,PF,77.94,light rain,-23.1203,-134.9692,
10,Saint-Philippe,RE,75.20,light rain,-21.3585,55.7679,
29,Warri,NG,79.90,broken clouds,5.5167,5.7500,
30,Butaritari,KI,82.22,overcast clouds,3.0707,172.7902,
31,Puerto Ayora,EC,80.01,light rain,-0.7393,-90.3518,
32,Brazzaville,CG,75.20,light rain,-4.2658,15.2832,
33,Honavar,IN,81.97,scattered clouds,14.2833,74.4500,
38,Nioro,GM,84.83,overcast clouds,13.3500,-15.7500,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [23]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,San Vicente,AR,76.10,moderate rain,-26.6167,-54.1333,Complejo Turístico Farelluk
3,Vaini,TO,75.20,few clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Rikitea,PF,77.94,light rain,-23.1203,-134.9692,Pension Maro'i
10,Saint-Philippe,RE,75.20,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
29,Warri,NG,79.90,broken clouds,5.5167,5.7500,BON Hotel Delta
...,...,...,...,...,...,...,...
688,Sumbe,AO,80.51,few clouds,-11.2061,13.8437,Hotel Ritz Sumbe
694,Kismayo,SO,83.03,overcast clouds,-0.3582,42.5454,Agan-Hotel
695,Nakhon Phanom,TH,76.12,few clouds,17.3333,104.4167,ป่ายางรีสอร์ท
696,Crixas,BR,84.07,overcast clouds,-14.5489,-49.9692,RISO HOTEL


In [25]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   225
Country                225
Max Temp               225
Current Description    225
Lat                    225
Lng                    225
Hotel Name             225
dtype: int64

In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))